In [ ]:
import pandas as pd
from os.path import dirname, isfile, join
import numpy as np


In [ ]:
# read simulation scenarios
mdir = r"../../3_models/sfincs"
rdir = r"../../4_results"
scens = pd.read_csv(join(rdir, 'sim_SCEN.csv'), index_col=0).rename(columns={'h_tsw_rp': 'h_rp'})

In [ ]:
from hydromt_sfincs import SfincsModel
mod0 = SfincsModel(join(mdir, '00_base_riv'), mode='r')
rivmsk = mod0.staticmaps['rivmsk'].raster.flipud()==1

In [ ]:
# read results
import xarray as xr

da_lst, drop_lst = [], []
for i, scen in scens.iterrows():
    name = scen['scen']
    root = join(mdir, name)
    fn_hmax = join(root, 'gis', 'hmax.tif')
    if isfile(fn_hmax):
        da_lst.append(xr.open_rasterio(fn_hmax, chunks='auto').squeeze(drop=True))
    else:
        drop_lst.append(i)



In [ ]:
# combine results

da_hmax = xr.concat(da_lst, dim='index').rename('hmax')
da_hmax.raster.set_nodata(-999)
da_hmax.raster.set_crs(mod0.crs)
da_hmax['index'] = xr.IndexVariable('index', scens.index.values)
da_meta = scens.drop(drop_lst).to_xarray()
da_hmax = xr.merge([da_hmax, da_meta]).set_coords(da_meta.data_vars)['hmax']
da_hmax = da_hmax.swap_dims({'index':'scen'})

In [ ]:
# correct for permanent water 
# & bias correct for Q but make sure to preserve P
nodata = da_hmax.raster.nodata
# get Q2 + MSHW map for bias correction
bias, da0 = 'q2', da_hmax.sel(scen=['qb002_qp002_h000_p000']).squeeze(drop=True)
da0.raster.flipud().raster.to_raster(join(rdir, 'hmax', f'bias_{bias}.tif'))
mask = np.logical_and(da0!=nodata, ~rivmsk)
# preserve P in all scenarios
pscen = [f'qb000_qp000_h000_p{s[-3:]}' for s in da_hmax.scen.values]
dap = da_hmax.sel(scen=pscen).reset_coords(drop=True).drop('scen')
dap = np.maximum(0, dap - da_hmax.sel(scen=['qb000_qp000_h000_p000']).squeeze(drop=True))
# take maximum of pscen and bias corrected scens
da_hmax1 = np.maximum(dap, da_hmax-da0).where(mask, nodata)
da_hmax1.raster.set_nodata(nodata)

In [ ]:
da_hmax1.chunk({'scen':50, 'x':250, 'y':250}).to_dataset().to_zarr(join(rdir, 'hmax.zarr'))